In [1]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# import data
data = pd.read_csv('C:/Users/Austin/Desktop/OEAS895_Clayton/Project/Data/model_input.csv') #read in data
data.head()

,Date,Date_Julian,DMP_NoWater,Surface_Irradiance,Below_Ice_Irradiance,Surface_Temp_C,Surface_Temp_K,Joules,Lat,Lon,Days_Above_24hr_Frz_AirTemp,Hrs_Above_Frz_Ice_Since_Jun1,Cum_Surface_Irradiance_SinceJun1,Cum_Sub-Surface_Irradiance_SinceJun1,Source,Split
0,1-Jun,152,0.001,62300000.0,814000.0,1.689247,274.839247,3.790000e-21,72.844081,-166.537644,0.0,0,2.070000e+09,84000000.0,Buoy7,Train
1,4-Jun,155,0.005,65000000.0,1840000.0,3.694515,276.844515,3.820000e-21,73.037055,-166.767810,2.0,24,2.260000e+09,88300000.0,Buoy7,Test
2,5-Jun,156,0.013,64300000.0,1450000.0,4.947612,278.097612,3.840000e-21,72.844835,-163.319440,1.0,60,2.460000e+09,45700000.0,Buoy8,Train
3,5-Jun,156,0.017,58700000.0,2520000.0,5.777930,278.927930,3.850000e-21,73.055408,-166.908423,3.0,41,2.320000e+09,90800000.0,Buoy7,Train
4,10-Jun,161,0.020,133000000.0,349000.0,-1.576667,271.573333,3.750000e-21,76.865833,-165.986000,0.0,0,6.020000e+09,4410000.0,SHEBA,Train


In [3]:
#define variables for machine learning
test=data[~data.Split.str.contains("Train")] #create test data
train=data[~data.Split.str.contains("Test")] #create train data

x_train=train[['Hrs_Above_Frz_Ice_Since_Jun1','Lat','Lon','Cum_Surface_Irradiance_SinceJun1']] #slight increase in r^2 (0.02)
x_test=test[['Hrs_Above_Frz_Ice_Since_Jun1','Lat','Lon','Cum_Surface_Irradiance_SinceJun1']]

#set target variable
y_train=train[['DMP_NoWater']]
y_test=test[['DMP_NoWater']]
y_train=np.ravel(y_train)
y_test=np.ravel(y_test)
y=data.DMP_NoWater

print(np.shape(x_test))
print(np.shape(x_train))
print(np.shape(y_test))
print(np.shape(y_train))

(8, 4)
(14, 4)
(8,)
(14,)


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

C:\Users\Austin\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Austin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
C:\Users\Austin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [5]:
##EXPLORE PARAMETER SPACE

#import sklearn packages
from sklearn.neural_network import MLPRegressor
from sklearn import metrics

#create empty lists
scores=[]
scores_list=[]
results=[]

#create a range of numbers of nodes to be tested (10-100, jumping by 10's)
n = tuple(range(10, 101, 10))
print(n)
#create a range of numbers of layers to be tested (1-3)
l = tuple(range(1,4,1))
print(l)

#build a nested loop to loop through both various numbers of layers and nodes
for l in l:
    layers = np.ones(l, dtype=np.int8) #create an array of ones that loops with l
    print(layers)
    for i in n:
        model = MLPRegressor(hidden_layer_sizes=(i*layers), max_iter=1000, random_state=1) #1000 iterations wasn't enough to optimize
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores = metrics.r2_score(y_test,y_pred)
        scores1 = metrics.mean_squared_error(y_test,y_pred)
        print(l,i,scores, scores1)
        scores_list.append(metrics.r2_score(y_test,y_pred))
        
#printed results show layer, nodes, r^2, and MSE

(10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
(1, 2, 3)
[1]
1 10 -2.4936407528872784 0.029725587757804643
1 20 -0.30272669495767746 0.01108422972894279
1 30 0.10319839052018165 0.007630422481733887
1 40 0.8838559421676191 0.0009882099013159098
1 50 0.7112974483724246 0.0024564211495458556
1 60 0.31596898293336606 0.0058200672207268135
1 70 0.9023011138002034 0.0008312694466858726
1 80 0.8187647221554143 0.0015420375297394411
1 90 0.7798646827489806 0.0018730179072159667
1 100 0.8031378882705011 0.0016749982016799453
[1 1]
2 10 -0.6795871619048648 0.014290741123518138
2 20 0.5063991140039421 0.004199795425983615
2 30 0.5567716930583428 0.003771201124170794
2 40 0.7954603824312769 0.0017403221401519552
2 50 0.6790851823376747 0.0027304987117858694
2 60 0.783971516021297 0.0018380749804877325


C:\Users\Austin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:302: RuntimeWarning: overflow encountered in byte_scalars
  init_bound = np.sqrt(factor / (fan_in + fan_out))
C:\Users\Austin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:302: RuntimeWarning: invalid value encountered in sqrt
  init_bound = np.sqrt(factor / (fan_in + fan_out))


OverflowError: Range exceeds valid bounds